# 📝 Text Summarization — BART Fine-tuning on SAMSum

This notebook trains **facebook/bart-large-cnn** on the **Samsung SAMSum processed** dataset
([koushik7198/Samsung-samsum_processed](https://huggingface.co/datasets/koushik7198/Samsung-samsum_processed)) —
a **public** dataset, no authentication needed.

**Before running:**
1. **Enable GPU**: Kaggle → Settings → Accelerator → **GPU T4 x2** (or Colab → Runtime → T4 GPU)

**Training time:** ~2 hours on T4 GPU (3 epochs)

## 1. Clone Repository from GitHub

In [ ]:
import os

REPO_URL = "https://github.com/saksham-1304/Text-Summarization-using-NLP.git"
REPO_DIR = "Text-Summarization-using-NLP"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}
    print(f"Cloned {REPO_URL}")
else:
    !cd {REPO_DIR} && git pull
    print(f"Pulled latest from {REPO_URL}")

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

## 2. Install Dependencies

In [ ]:

# Install compatible versions
!pip install -q "transformers>=4.41.0" "datasets>=2.16.1" "evaluate>=0.4.1" \
    rouge-score accelerate sentencepiece protobuf \
    huggingface_hub python-box pyyaml pandas pyarrow

# ── Free pip cache immediately to reclaim ~1-2 GB ────────────────────────────
!pip cache purge -q

# Suppress frozen modules debugger warning
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

# Show available disk space
import shutil
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Dependencies installed. Disk: {free/1e9:.1f} GB free / {total/1e9:.1f} GB total")


## 3. Verify GPU Availability

In [ ]:

import os
import torch

# ── Pin both Kaggle T4 GPUs ───────────────────────────────────────────────────
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

print(f"PyTorch version:  {torch.__version__}")
print(f"CUDA available:   {torch.cuda.is_available()}")
print(f"GPUs available:   {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        free, total = torch.cuda.mem_get_info(i)
        print(f"\n  GPU {i}: {props.name}")
        print(f"    Total VRAM : {total / 1e9:.2f} GB")
        print(f"    Free  VRAM : {free  / 1e9:.2f} GB")

    total_vram = sum(torch.cuda.get_device_properties(i).total_memory
                     for i in range(torch.cuda.device_count()))
    print(f"\n  Combined VRAM : {total_vram / 1e9:.2f} GB")
    print("\nHuggingFace Trainer will use ALL GPUs via DataParallel automatically.")
else:
    print("WARNING: No GPU detected! Training will be extremely slow.")
    print("  Kaggle: Settings -> Accelerator -> GPU T4 x2")
    print("  Colab:  Runtime -> Change runtime type -> T4 GPU")


## 4. Load SAMSum Dataset

Using the **public** dataset `koushik7198/Samsung-samsum_processed` — no token required.

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

BASE = "hf://datasets/koushik7198/Samsung-samsum_processed/"
SPLITS = {
    "train":      "data/train-00000-of-00001.parquet",
    "validation": "data/validation-00000-of-00001.parquet",
    "test":       "data/test-00000-of-00001.parquet",
}

dataset = DatasetDict()
for split, path in SPLITS.items():
    df = pd.read_parquet(BASE + path)
    # Rename columns: input → dialogue, output → summary
    df = df.rename(columns={"input": "dialogue", "output": "summary"})
    dataset[split] = Dataset.from_pandas(df, preserve_index=False)

print(f"Train:      {len(dataset['train'])} examples")
print(f"Validation: {len(dataset['validation'])} examples")
print(f"Test:       {len(dataset['test'])} examples")
print(f"\nColumns: {dataset['train'].column_names}")
print(f"\n--- Example ---")
print(f"Dialogue: {dataset['train'][0]['dialogue'][:200]}...")
print(f"Summary:  {dataset['train'][0]['summary']}")

## 5. Load Tokenizer & Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

print(f"Model: {MODEL_NAME}")
print(f"Model parameters: {model.num_parameters() / 1e6:.0f}M")
print(f"Vocab size: {tokenizer.vocab_size}")

In [ ]:

# ── Model Optimizations ───────────────────────────────────────────────────────
# NOTE: Do NOT manually DataParallel-wrap here — HuggingFace Trainer handles
#       multi-GPU distribution internally. Manual wrapping causes double-wrap errors.

# 1. Move model to primary GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model loaded on : {next(model.parameters()).device}")
print(f"GPUs available  : {torch.cuda.device_count()}")
if torch.cuda.device_count() > 1:
    print(f"Trainer will distribute across all {torch.cuda.device_count()} GPUs via DataParallel automatically.")

# 2. Set model to training mode
model.train()
print(f"Model mode      : training")

# 3. Mixed precision sanity check
if torch.cuda.is_available():
    major, minor = torch.cuda.get_device_capability(0)
    fp16_ok = major >= 7          # Volta and newer support FP16 natively
    print(f"\nGPU compute capability : {major}.{minor}  →  FP16 {'✓ supported' if fp16_ok else '✗ not recommended'}")

# 4. Show per-GPU memory footprint after model load
if torch.cuda.is_available():
    print()
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1e9
        reserved  = torch.cuda.memory_reserved(i)  / 1e9
        free, total = torch.cuda.mem_get_info(i)
        print(f"GPU {i} memory after model load:")
        print(f"  Allocated : {allocated:.2f} GB")
        print(f"  Reserved  : {reserved:.2f} GB")
        print(f"  Free      : {free/1e9:.2f} GB / {total/1e9:.2f} GB")
        print(f"  Available : {(free/total*100):.1f}% free")


## 6. Tokenize Dataset

In [ ]:
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128

def preprocess_function(examples):
    """Tokenize dialogues and summaries."""
    inputs = examples["dialogue"]
    targets = examples["summary"]
    
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    labels = tokenizer(
        text_target=targets,
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing"
)

print(f"Tokenized train: {len(tokenized_dataset['train'])}")
print(f"Tokenized validation: {len(tokenized_dataset['validation'])}")
print(f"Tokenized test: {len(tokenized_dataset['test'])}")

## 7. Training Configuration

In [ ]:

import os
import shutil
import torch
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
import evaluate

# ── OOM + disk space fixes ─────────────────────────────────────────────────────
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

NUM_GPUS = torch.cuda.device_count() if torch.cuda.is_available() else 1
print(f"Training on {NUM_GPUS} GPU(s)")

# ── Disk space check before training ─────────────────────────────────────────
total, used, free = shutil.disk_usage("/kaggle/working")
print(f"Disk before training: {free/1e9:.1f} GB free / {total/1e9:.1f} GB total")
if free / 1e9 < 5:
    print("WARNING: Less than 5 GB free! Training may crash with 'No space left on device'.")
    print("  → Delete unused files or reduce save_total_limit further.")

# ROUGE metric — only used at test time, NOT during training eval
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Compute ROUGE — called only during final test evaluation, not mid-training."""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}


OUTPUT_DIR = "./bart-samsum-finetuned"

PER_DEVICE_TRAIN_BS = 2
PER_DEVICE_EVAL_BS  = 1
GRAD_ACCUM          = 8

# ~847 total steps → 10% warmup
ESTIMATED_TOTAL_STEPS = 850
WARMUP_STEPS = max(50, ESTIMATED_TOTAL_STEPS // 10)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BS,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=WARMUP_STEPS,

    # Evaluation
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,

    # ── DISK FIX: only keep 1 checkpoint at a time ───────────────────────────
    # Each checkpoint = ~1.6 GB. With 5 checkpoints = 8 GB → disk full crash.
    # Keep only the single best checkpoint to stay within 20 GB Kaggle limit.
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # No logit accumulation during training evals (OOM fix)
    prediction_loss_only=True,

    # Performance
    fp16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    dataloader_num_workers=0,   # avoids Kaggle fork/deadlock crash

    # Logging
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=50,
    report_to="none",
)

eff_batch = PER_DEVICE_TRAIN_BS * NUM_GPUS * GRAD_ACCUM
print("\nTraining configuration:")
print(f"  GPUs:                     {NUM_GPUS}")
print(f"  Effective batch size:     {eff_batch}  ({PER_DEVICE_TRAIN_BS} × {NUM_GPUS} GPUs × {GRAD_ACCUM})")
print(f"  Learning rate:            {training_args.learning_rate}")
print(f"  Warmup steps:             {WARMUP_STEPS}  (~10% of ~{ESTIMATED_TOTAL_STEPS})")
print(f"  FP16:                     {training_args.fp16}")
print(f"  Eval/Save every:          {training_args.eval_steps} steps")
print(f"  save_total_limit:         1  ← only 1 checkpoint kept (~1.6 GB max)")
print(f"  prediction_loss_only:     True  ← no logit gather during eval")
print(f"  dataloader_num_workers:   0  ← no forked workers")
print(f"  PYTORCH_CUDA_ALLOC_CONF:  {os.environ['PYTORCH_CUDA_ALLOC_CONF']}")


## 8. Train the Model

In [ ]:

import os
import glob
import shutil
import torch

# ── Check if the model was already saved from a previous run ─────────────────
FINAL_SAVE_DIR = "./bart-samsum-final"
prior_save_exists = os.path.exists(os.path.join(FINAL_SAVE_DIR, "config.json"))
if prior_save_exists:
    size = sum(os.path.getsize(os.path.join(FINAL_SAVE_DIR, f))
               for f in os.listdir(FINAL_SAVE_DIR)) / 1e9
    print(f"Found existing saved model at {FINAL_SAVE_DIR} ({size:.2f} GB)")
    print("Skip to cell 11 (Save) to re-save, or cell 10 (Inference) to test it.")
    print("Set FORCE_RETRAIN = True below to retrain anyway.")
    FORCE_RETRAIN = False
else:
    FORCE_RETRAIN = True

if FORCE_RETRAIN or not prior_save_exists:
    # ── Show per-GPU free memory before training ──────────────────────────────
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        for i in range(torch.cuda.device_count()):
            free, total = torch.cuda.mem_get_info(i)
            print(f"GPU {i} free: {free/1e9:.2f} GB / {total/1e9:.2f} GB")

    disk_free = shutil.disk_usage("/kaggle/working").free / 1e9
    print(f"Disk free before training: {disk_free:.1f} GB")

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    # ── Auto-resume from latest checkpoint ───────────────────────────────────
    search_paths = [
        training_args.output_dir,
        os.path.join("/kaggle/working", "bart-samsum-finetuned"),
        os.path.join("/kaggle/working", "Text-Summarization-using-NLP", "bart-samsum-finetuned"),
    ]
    resume_ckpt = None
    for search_dir in search_paths:
        ckpts = sorted(glob.glob(os.path.join(search_dir, "checkpoint-*")))
        if ckpts:
            resume_ckpt = ckpts[-1]
            break

    if resume_ckpt:
        print(f"\nResuming from checkpoint: {resume_ckpt}")
    else:
        print("\nNo checkpoint found — starting from scratch.")

    num_gpus = torch.cuda.device_count() if torch.cuda.is_available() else 1
    print(f"Starting training on {num_gpus} GPU(s)... (~2 hours on 2× T4)")
    train_result = trainer.train(resume_from_checkpoint=resume_ckpt)

    # ── Save best model ───────────────────────────────────────────────────────
    trainer.save_model(training_args.output_dir)

    disk_free_after = shutil.disk_usage("/kaggle/working").free / 1e9
    print(f"\nDisk free after training: {disk_free_after:.1f} GB")

    print(f"\n{'='*50}")
    print("Training Complete!")
    print(f"{'='*50}")
    for k, v in sorted(train_result.metrics.items()):
        print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")


## 9. Evaluate on Test Set

In [ ]:

# ── Test set ROUGE evaluation — batched generation (no RAM accumulation) ──────
import os
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM

# Resolve which model to use:
#   1. trainer.model  — if training ran this session
#   2. fine-tuned weights on disk  — if training ran in a previous session
#   3. base model  — last resort (results will be poor)
try:
    eval_model = trainer.model
    print("Using trainer.model (fine-tuned this session)")
except NameError:
    for candidate in ["./bart-samsum-final", "./bart-samsum-finetuned"]:
        if os.path.exists(os.path.join(candidate, "config.json")):
            print(f"Loading fine-tuned model from {candidate} ...")
            _dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            eval_model = AutoModelForSeq2SeqLM.from_pretrained(candidate).to(_dev)
            print("Fine-tuned model loaded successfully.")
            break
    else:
        print("WARNING: No fine-tuned model found — using base model. Results will be poor.")
        eval_model = model

eval_model.eval()
device = next(eval_model.parameters()).device
print(f"Eval device: {device}")

EVAL_BATCH_SIZE = 4    # reduce to 2 if this cell still OOMs

all_preds, all_refs = [], []

# Set format to torch so DataLoader returns tensors directly
tokenized_dataset["test"].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

test_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=EVAL_BATCH_SIZE,
    shuffle=False,
)

print(f"Evaluating {len(tokenized_dataset['test'])} test examples "
      f"in batches of {EVAL_BATCH_SIZE}...")

with torch.no_grad():
    for step, batch in enumerate(test_loader):
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_ids      = batch["labels"]   # CPU only

        generated = eval_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            num_beams=1,
            no_repeat_ngram_size=5,
            early_stopping=True,
        )

        decoded_preds  = tokenizer.batch_decode(generated.cpu(), skip_special_tokens=True)
        lab_np = label_ids.numpy().copy()
        lab_np[lab_np == -100] = tokenizer.pad_token_id
        decoded_labels = tokenizer.batch_decode(lab_np, skip_special_tokens=True)

        all_preds.extend(decoded_preds)
        all_refs.extend(decoded_labels)

        del input_ids, attention_mask, generated
        torch.cuda.empty_cache()

        if (step + 1) % 50 == 0:
            done = min((step + 1) * EVAL_BATCH_SIZE, len(tokenized_dataset["test"]))
            print(f"  [{done}/{len(tokenized_dataset['test'])}] done")

rouge_scores = rouge.compute(
    predictions=all_preds,
    references=all_refs,
    use_stemmer=True
)

print(f"\n{'='*50}")
print("Test Set ROUGE Scores (fine-tuned BART)")
print(f"{'='*50}")
for k in ["rouge1", "rouge2", "rougeL", "rougeLsum"]:
    print(f"  {k:12s}: {rouge_scores[k]:.4f}")
print(f"\n  Evaluated on {len(all_preds)} examples")


## 10. Test Inference

In [ ]:

import os
import torch
from transformers import AutoModelForSeq2SeqLM

# Resolve which model to use (same logic as eval cell)
try:
    infer_model = trainer.model
    print("Using trainer.model (fine-tuned this session)")
except NameError:
    for candidate in ["./bart-samsum-final", "./bart-samsum-finetuned"]:
        if os.path.exists(os.path.join(candidate, "config.json")):
            print(f"Loading fine-tuned model from {candidate} ...")
            _dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            infer_model = AutoModelForSeq2SeqLM.from_pretrained(candidate).to(_dev)
            print("Fine-tuned model loaded successfully.")
            break
    else:
        print("WARNING: No fine-tuned model found — using base model. Results will be poor.")
        infer_model = model

infer_model.eval()
infer_device = next(infer_model.parameters()).device

def summarize(text, max_new_tokens=60, num_beams=1):
    """Generate a summary using model.generate() directly (pipeline-free)."""
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding=True,
    )
    input_ids      = inputs["input_ids"].to(infer_device)
    attention_mask = inputs["attention_mask"].to(infer_device)

    with torch.no_grad():
        output_ids = infer_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=5,   # prevents repetitive / hallucinated phrases
            length_penalty=0.8,       # encourages shorter, tighter summaries
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


test_dialogues = [
    """Amanda: Hey, are we meeting today?
Jerry: Sure! What time works for you?
Amanda: How about 3pm at the coffee shop?
Jerry: Perfect, see you there!
Amanda: Great, I'll bring the project reports.""",

    """Sarah: Did you finish the report?
Mike: Almost done, just need to review the numbers.
Sarah: The deadline is tomorrow at noon.
Mike: I know, I'll send it tonight.
Sarah: Great, also include the Q3 projections.
Mike: Will do. Should I CC the manager?
Sarah: Yes, please CC David.""",

    """Tom: Happy birthday! 🎂
Lisa: Thank you so much! 😊
Tom: Are you doing anything special?
Lisa: Just a small dinner with family.
Tom: That sounds lovely. Enjoy your day!"""
]

print("=" * 60)
print("INFERENCE EXAMPLES")
print("=" * 60)

for i, dialogue in enumerate(test_dialogues, 1):
    summary = summarize(dialogue)
    print(f"\n--- Example {i} ---")
    print(f"Dialogue:\n{dialogue}")
    print(f"\nSummary: {summary}")
    print()


## 11. Save & Download Model

Save the fine-tuned model. On Kaggle, download from the **Output** tab.

In [ ]:

import os
import shutil

SAVE_DIR = "./bart-samsum-final"
os.makedirs(SAVE_DIR, exist_ok=True)

# Use trainer.model if available, otherwise load from the saved checkpoint dir
try:
    save_model = trainer.model
    save_model.save_pretrained(SAVE_DIR)
    tokenizer.save_pretrained(SAVE_DIR)
    print(f"Model saved from trainer → {SAVE_DIR}")
except NameError:
    # trainer was not run (model loaded from prior run) — copy from OUTPUT_DIR
    OUTPUT_DIR = "./bart-samsum-finetuned"
    if os.path.exists(OUTPUT_DIR):
        for f in os.listdir(OUTPUT_DIR):
            if not f.startswith("checkpoint"):
                shutil.copy2(os.path.join(OUTPUT_DIR, f), SAVE_DIR)
        tokenizer.save_pretrained(SAVE_DIR)
        print(f"Model copied from {OUTPUT_DIR} → {SAVE_DIR}")
    else:
        print("ERROR: No trained model found. Run the training cell first.")

# Show saved files and total size
total_size = 0
for f in sorted(os.listdir(SAVE_DIR)):
    size = os.path.getsize(os.path.join(SAVE_DIR, f))
    total_size += size
    print(f"  {f}: {size / 1e6:.1f} MB")

print(f"\nTotal model size: {total_size / 1e9:.2f} GB")

# Final disk status
free = shutil.disk_usage("/kaggle/working").free / 1e9
print(f"Disk remaining:   {free:.1f} GB free")
print("\nOn Kaggle: Go to the Output tab → download bart-samsum-final/")


In [ ]:

# ── Zip the model for easy single-file download ───────────────────────────────
import os
import zipfile
import shutil

SAVE_DIR  = "./bart-samsum-final"
ZIP_PATH  = "/kaggle/working/bart-samsum-final.zip"

if not os.path.exists(SAVE_DIR):
    print(f"ERROR: {SAVE_DIR} not found — run the Save cell first.")
else:
    print(f"Zipping {SAVE_DIR} → {ZIP_PATH} ...")
    with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as zf:
        for fname in sorted(os.listdir(SAVE_DIR)):
            fpath = os.path.join(SAVE_DIR, fname)
            zf.write(fpath, arcname=os.path.join("bart-samsum-final", fname))
            print(f"  added: {fname}  ({os.path.getsize(fpath)/1e6:.1f} MB)")

    zip_size = os.path.getsize(ZIP_PATH) / 1e9
    free     = shutil.disk_usage("/kaggle/working").free / 1e9
    print(f"\nZip created : {ZIP_PATH}")
    print(f"Zip size    : {zip_size:.2f} GB")
    print(f"Disk free   : {free:.1f} GB")
    print("\nTo download on Kaggle:")
    print("  Output tab (right panel) → find bart-samsum-final.zip → click Download")


---

## After Training

1. **Download** the `bart-samsum-final/` folder
2. **Copy** it to your project at `artifacts/model_trainer/bart-samsum-model/`
3. **Run** the API server: `python app.py`
4. **Open** `http://localhost:8080` for the web UI

### Expected Results (3 epochs)

| Metric | Score |
|--------|-------|
| ROUGE-1 | ~0.52 |
| ROUGE-2 | ~0.28 |
| ROUGE-L | ~0.43 |
| ROUGE-Lsum | ~0.48 |